In [15]:
def main():
    """
  This function is the main entry point for the script.
  It performs the following tasks:
    1. Authenticates with IBM Cloud using an API key.
    2. Loads testing data from an Excel file.
    3. Prepares the data for prediction.
    4. Creates a payload with the data for the Watson Machine Learning model.
    5. Sends a POST request to the deployed model for prediction on the testing data.
    7. Reformats the prediction results and adds them to the "sales" column of the testing data.
    8. Exports the data with predictions to a new Excel file.
  """
    import requests
    import pandas as pd

    # Replace with your actual API key 
    API_KEY = "YOUR_API_KEY"

    # Authentication with IBM Cloud
    token_response = requests.post('https://iam.cloud.ibm.com/identity/token', data={"apikey":
     API_KEY, "grant_type": 'urn:ibm:params:oauth:grant-type:apikey'})
    
    mltoken = token_response.json()["access_token"]

    header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}
    # Assuming 'column_names_list' is used later for prediction
    
    df_col = pd.read_csv('/kaggle/input/excel-sales-dataset-training-and-testing/ExcelSalesForecast/Training_Dataset.csv')
    
    # Load testing data
    df = pd.read_excel("/kaggle/input/excel-sales-dataset-training-and-testing/ExcelSalesForecast/Store Sales.xlsx")
    
    column_names_list = df.columns.tolist()
    colum_names_list_except_sales_col = column_names_list[:-1]
    df['Date'] = df['Date'].astype(str)
    df.fillna(0, inplace = True) #Replace missing values with 0
    
    # Prepare payload for prediction
    payload_scoring = {"input_data": [{"fields": colum_names_list_except_sales_col, "values": df.values.tolist()}]}
    
    # Send prediction request to deployed model
    response_scoring = requests.post('https://eu-de.ml.cloud.ibm.com/ml/v4/deployments/27d6a8fc-c18c-4312-9837-780d03901cb8/predictions?version=2021-05-01', json=payload_scoring,
    headers={'Authorization': 'Bearer ' + mltoken})
    
    # Extract predictions from response
    predictions = response_scoring.json()
    
    # Add predictions to 'Sales' column of testing dataset
    df['Sales'] = [x[0] for x in predictions ['predictions'] [0] ['values']] 

    # Export data with predictions to Excel
    df.to_excel ("Predicted Sales.xlsx")


In [16]:
if __name__ =="__main__":
    main()